In [1]:
import pandas as pd


from sklearn.utils.validation import column_or_1d

path =  '/data/public/MLA/VCHAMPS/measurements_train.csv'
data = pd.read_csv(path).iloc[:,1:]

In [5]:
data

,Internalpatientid,Age at measurement,Measurement date,Measurement,Result numeric,Result textual,State
0,1,59.124538,2003-05-21 00:27:01.0,Temperature,95.804066,NaN,Indiana
1,1,59.124538,2003-05-21 00:27:01.0,Pain,3.000000,NaN,Indiana
2,1,59.124538,2003-05-21 00:27:01.0,Pulse,74.000000,NaN,Indiana
3,1,59.124538,2003-05-21 00:27:01.0,Respiratory rate,16.000000,NaN,Indiana
4,1,59.124538,2003-05-21 00:27:01.0,Blood pressure,NaN,140/96,Indiana
...,...,...,...,...,...,...,...
125247157,99999,96.341924,2013-04-03 23:21:35.0,Pulse oximetry,94.000000,NaN,Arizona
125247158,99999,96.341924,2013-04-03 23:21:35.0,Temperature,92.336347,NaN,Arizona
125247159,99999,96.341924,2013-04-03 23:21:35.0,Blood pressure,NaN,141/49,Arizona
125247160,99999,96.341924,2013-04-03 23:21:35.0,Respiratory rate,36.000000,NaN,Arizona
